# Trevor Maxwell
## Sports Radar API
### d20230727

Data that will be pulled from API is the player's place on the depth chart (starter, back up, third string, etc.,).

In [1]:
# import necessary libraries

import pandas as pd
import numpy as np
import json
import requests
import time

In [2]:
# assign api key,api type, version, language, and season type to api url

api_key = 'your_API_key_here'
api_type = 'trial'
version = 'v7'
lang_code = 'en'
season_type = 'REG'
request_type = 'json'
week_number = 1

# assign year and week_number for testing, eventually will loop through 2006 - 2016 for all 16 weeks of the season
year = 2008


url = f"http://api.sportradar.us/nfl/official/{api_type}/{version}/{lang_code}/seasons/{str(year)}/{season_type}/{str(week_number)}/depth_charts.{request_type}?api_key={api_key}"

In [3]:
# create player dictionary that will become dataframe from API
player_dict = {'id':[], 'team':[], 'year':[], 'name':[], 'position':[], 'depth':[]}

In [4]:
# make request to api from 2006 to 2016 and obtain depth chart data from api
for year in range(2006, 2017):
    
    # delay each loop by one second to avoid making requests too quickly to API
    time.sleep(1)
    
    # define url to pass to GET request
    url = f"http://api.sportradar.us/nfl/official/{api_type}/{version}/{lang_code}/seasons/{year}/{season_type}/{str(week_number)}/depth_charts.{request_type}?api_key={api_key}"

    try:
        # Make GET request
        temp_dict = requests.get(url)

    except :
        # print if error occurred
        print(f'Error Occurred with year {year}')

    else:
        # turn response to JSON text
        temp_dict = temp_dict.text
        temp_dict_text = json.loads(temp_dict)
        
        # create dictionary of team data
        teams_dict = temp_dict_text["teams"]
        
        # loop through all 32 teams
        for team in range(0, 31):
            
            # assign alias to team abbreviation from dictionary
            alias = temp_dict_text["teams"][team]["alias"]
    #         player_dict['Tm'].append(alias)

            # loop through all positions on offense
            for x in range(len(teams_dict[team]['offense'])-1):
    
                # if the position in the loop is the runningback position
                if teams_dict[team]['offense'][x]['position']['name'] == 'RB':
                    
                    # loop through all listed runningbacks
                    for num_players in range(len(teams_dict[team]['offense'][x]['position']['players'])):

                        # assign runningbacks's dictionary to player variable
                        player = teams_dict[team]['offense'][x]['position']['players'][num_players]

                        # append runningback's data to player dictionary
                        player_dict['id'].append(player['id'])
                        player_dict['team'].append(alias)
                        player_dict['year'].append(year)
                        player_dict['name'].append(player['name'])
                        player_dict['position'].append(player['position'])
                        player_dict['depth'].append(player['depth'])

                    # append runningback's data to the dataframe
                    depth_df = pd.DataFrame(player_dict)

In [5]:
# view preview of dataframe
depth_df.head(20)

,id,team,year,name,position,depth
0,35f295ed-963f-45eb-983b-c279c7b0827c,ARI,2006,Edgerrin James,RB,1
1,6b4bba83-9f7e-4039-ae3b-1d4f5783c1d0,ARI,2006,J.J. Arrington,RB,2
2,69119df7-c315-4983-9354-58d4c031b8ac,ARI,2006,Diamond Ferri,RB,3
3,481bc72e-d062-4e90-bedb-c31066c8515a,ARI,2006,Marcel Shipp,RB,3
4,46806836-3b8f-41c5-827f-98f20de2a46a,ATL,2006,Warrick Dunn,RB,1
5,44f7aa7d-931a-40d6-a2dc-7b798570e11a,ATL,2006,Jerious Norwood,RB,2
6,223d4b04-ba22-47fc-a435-52f106f7a126,BAL,2006,Jamal Lewis,RB,1
7,9e667c7c-c1c0-477c-a5be-4fa9b3a4b3f3,BAL,2006,Mike Anderson,RB,2
8,cbeb4274-49d7-43a6-afb4-9c9f28615645,BAL,2006,P.J. Daniels,RB,3
9,060e5a7c-9a98-4a07-a7f3-1385493a3598,CAR,2006,DeShaun Foster,RB,1


## Cleaning

#### Cleaning Step 1: making first and last name columns
 - The name needs to be separated by first name and last name because the name will be used to match to the records from the other sources.

In [6]:
# separate first name from last name and create new columns

depth_df[["first_name", "last_name"]] = depth_df["name"].str.split(" ", n = 1, expand = True)

#### Cleaning Step 2: dropping original name column
 - Keeping the old column is redundant could lead to confusion having the same data in different columns.

In [7]:
# drop old name column

depth_df1 = depth_df.drop(columns = ['name'], axis = 1)

In [8]:
# confirm dropped column and new added columns
depth_df1.head()

,id,team,year,position,depth,first_name,last_name
0,35f295ed-963f-45eb-983b-c279c7b0827c,ARI,2006,RB,1,Edgerrin,James
1,6b4bba83-9f7e-4039-ae3b-1d4f5783c1d0,ARI,2006,RB,2,J.J.,Arrington
2,69119df7-c315-4983-9354-58d4c031b8ac,ARI,2006,RB,3,Diamond,Ferri
3,481bc72e-d062-4e90-bedb-c31066c8515a,ARI,2006,RB,3,Marcel,Shipp
4,46806836-3b8f-41c5-827f-98f20de2a46a,ATL,2006,RB,1,Warrick,Dunn


#### Cleaning Step 3: Dropping ID column
 - The ID column was grabbed from the API in case it could be used to match to the data from the other sources, but the other sources do not have the same IDs for the players, so it can be removed.

In [9]:
# drop id column, it is not needed

depth_df2 = depth_df1.drop(columns = ['id'], axis = 1)

In [10]:
# confirm dropped column
depth_df2.head()

,team,year,position,depth,first_name,last_name
0,ARI,2006,RB,1,Edgerrin,James
1,ARI,2006,RB,2,J.J.,Arrington
2,ARI,2006,RB,3,Diamond,Ferri
3,ARI,2006,RB,3,Marcel,Shipp
4,ATL,2006,RB,1,Warrick,Dunn


#### Cleaning Step 5: Exporting to CSV
 - Exporting the data to CSV will allow easy access to the depth chart data and avoid sending too many requests to the API.

In [13]:
# export to csv

depth_df2.to_csv('depth_chart_data.csv', index = None)